# EM Algorithm for Clustering and Missing values

## About this notebook: 
Notebook prepared by **Jesus Perez Colino** Version 0.1, First Released: 21/03/2018, Alpha.  

- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: This notebook is just the simplest implementation of the **EM-algorithm** with full description. The objective is not to write the fastest implementation (will come later) but the best possibly explained.


- TODO: will be extended soon to vectorized version

In [1]:
from sys import version 
import IPython
import pandas as pd
import numpy as np
import scipy.stats as scs
from scipy.stats import multivariate_normal as mvn
from scipy.optimize import minimize

print (' EM-Algo: Reproducibility conditions for this notebook '.center(85,'-'))
print ('Python version:     ' + version)
print ('Numpy version:      ' + np.__version__)
print ('IPython version:    ' + IPython.__version__)
print ('-'*85)

--------------- EM-Algo: Reproducibility conditions for this notebook ---------------
Python version:     3.5.5 |Anaconda custom (64-bit)| (default, Mar 12 2018, 16:25:05) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Numpy version:      1.14.2
IPython version:    6.2.1
-------------------------------------------------------------------------------------


## Introduction

In this notebook, we study the maximum likelihood estimation by the **EM algorithm**, a special case of the **MM algorithm **(M stands for minorize and the second M for maximize). At the heart of every **EM algorithm** is some notion of missing data. Data can be missing in the ordinary sense, such as a failure to record certain observations on certain cases, or also  data can also be missing in a theoretical sense. In any case, we can think of the **E-step** (expectation) of the algorithm as filling in the missing data. This action replaces the loglikelihood of the observed data by a minorizing function, which is then maximized in the **M-step**. Because the surrogate function is usually much simpler than the likelihood, we can often solve the **M-step** analytically. The price we pay for this simplification is *iteration*. Reconstruction of the missing data is bound to be slightly wrong if the parameters do not already equal their maximum likelihood estimates.

One of the advantages of the **EM algorithm** is its numerical stability. As an MM algorithm, any **EM algorithm** leads to a steady increase in the likelihood of the observed data. Thus, the **EM algorithm** avoids wildly overshooting or undershooting the maximum of the likelihood along its current direction of search. Besides this desirable feature, the **EM algorithm** handles parameter constraints gracefully. Constraint satisfaction is by definition built into the solution of the M step. In contrast, competing methods of maximization must employ special techniques to cope with parameter constraints. The **EM algorithm** shares some of the negative features of the more general MM algorithm. For example, the **EM algorithm** often converges at an excruciatingly slow rate in a neighborhood of the maximum point.

Examples of applications of the **EM algorithm** include:

- Filling in missing data from a sample set
- Discovering values of latent variables
- Estimating parameters of HMMs
- Estimating parameters of finite mixtures [models]
- Unsupervised learning of clusters
- etc...

Let us begin with the simplest case...

## Maximum likelihood with complete information
Consider an experiment with coin $A$ that has a probability $\theta_A$ of heads, and a coin $B$ that has a probability $\theta_B$ of tails. 

We draw $m$ samples as follows - for each sample, pick one of the coins at random, flip it $n$ times, and record the number of heads and tails (that sum to $n$). If we recorded which coin we used for each sample, we have complete information and can estimate $\theta_A$ and $\theta_B$ in closed form. 

To be more specific, let us assume we drew $5$ samples with the number of heads and tails represented as a vector $x$, and the sequence of coins chosen was $A,A,B,A,B$. Then the complete log-likelihood function is

$$\log L(\theta;\mathbf{x}) = \log p(x_1;\theta_A)+\log p(x_2;\theta_A)+ \log p(x_3;\theta_B)+\log p(x_4;\theta_A)+\log p(x_5;\theta_B)$$


where $p(x_i;\theta)$ is the binomial distribtion PMF with $n=m$ and $p=\theta$. We will use $z_i$ to indicate the label of the $i$-th coin, that is - whether we used coin $A$ or $B$ to gnerate the $i$-th sample.

In [2]:
# Let's simulate the experiment in Python

m = 10        # number of trials
theta_A = 0.8  # probability of HEADS in coin A
theta_B = 0.3  # probability of HEADS in coin B
theta_0 = [theta_A, theta_B]

coin_A = scs.bernoulli(theta_A)
coin_B = scs.bernoulli(theta_B)

experiment = np.fromiter(map(sum, [coin_A.rvs(m), 
                                   coin_A.rvs(m), 
                                   coin_B.rvs(m), 
                                   coin_A.rvs(m), 
                                   coin_B.rvs(m)]),np.int)

coins_sequence = [0, 0, 1, 0, 1] # A, A, B, A, B

In [3]:
coin_A.rvs(m)

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1])

In [4]:
# 5 experiments with 10 trials each where we note as 1:HEAD and 0:TAIL
# where we are using the sequence of coins: A, A, B, A, B

experiment

array([8, 9, 4, 9, 3])

In [5]:
maxlikelihood_A = np.sum(experiment[[0,1,3]])/(3.0*m)
maxlikelihood_B = np.sum(experiment[[2,4]])/(2.0*m)
maxlikelihood_A, maxlikelihood_B


(0.8666666666666667, 0.35)

In [6]:
def neg_loglikelihood(thetas, n, experiment, coins_sequence):
    return -np.sum([scs.binom(n, thetas[j]).logpmf(i) for (i, j) 
                    in zip(experiment, coins_sequence)])

In [7]:
bnds = [(0,1), (0,1)]
minimize(neg_loglikelihood, 
         [0.5, 0.5], 
         args=(m, experiment, coins_sequence),
         bounds=bnds, 
         method='tnc', 
         options={'maxiter': 100})

     fun: 6.182734848389383
     jac: array([ 9.20152843e-05, -1.40154555e-04])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 55
     nit: 10
  status: 1
 success: True
       x: array([0.86666702, 0.3499984 ])

## Maximum likelihood with Incomplete information

Let assume now that we keep the result of the previous experiment, head or tail, and we store it in the variable $\mathbf X$, but assume that we did not record the coin we used. Now we have **missing data**, defined as $\mathbf Z$, and the problem of estimating $\mathbf \theta$ is harder to solve. One way to approach the problem is to ask - can we assign weights $z_j$ defined as $z_j = P(X \in j)$ for $ j = {A, B}$ to each sample according to how likely it is to be generated from coin $A$ or coin $B$?

With some *'prior'* knowledge of $z_j$, we can maximize the likelihod to find $\theta$. Similarly, given $z_j$, we can calculate what $\theta$ should be. So the basic idea behind **Expectation Maximization (EM)** is simply to start with a guess for $\theta$, then calculate the log-likelihood function of the sample, then update $\theta$ using the value that maximizes the previous log-likelihood function, and *repeat till convergence*. 

In general, any estimation problem where the **EM-algorithm** has an useful role, has a similar framework: given the statistical experiment/model which generates a set $\mathbf{X}$ of observed data, a set of unobserved data or missing values $\mathbf{Z}$, and a vector of unknown parameters $\boldsymbol\theta$, along with a likelihood function:

$$L(\boldsymbol\theta; \mathbf{X}, \mathbf{Z}) = p(\mathbf{X}, \mathbf{Z}|\boldsymbol\theta)$$

the maximum likelihood estimate (MLE) of the unknown parameters is determined by the marginal likelihood of the observed data:

$$L(\boldsymbol\theta; \mathbf{X}) = p(\mathbf{X}|\boldsymbol\theta) = \int  p(\mathbf{X},\mathbf{Z}|\boldsymbol\theta) d\mathbf{Z}$$

However, this quantity is often intractable (e.g. if $\mathbf{Z}$ is a sequence of events, so that the number of values grows exponentially with the sequence length, making the exact calculation of the sum extremely difficult).

The **EM algorithm** seeks to find a maximum likelihood estimation of the marginal likelihood by iteratively applying the **E** (expectation) and **M** (maximization) steps. The derivation below shows why the **EM algorithm** using these “alternating” updates actually works: 

- *First*, given the current estimate of the parameters $\theta^{(t)}$, where $t$ is a counting variable of the iteration, consider the log-likelihood function as a curve (surface) where the base is $\theta$. The idea start finding another function ${\displaystyle Q(\theta | \theta^{(t)})=\mathbb{E}[\log L(X |\theta) | Y=y, \theta^{(t)}]}$, where $y$ is the actual observed data (incomplete) and $\theta^{(t)}$ is the current estimated value of $\theta$ (**E-step**)

$$\begin{align}Q(\theta|\theta_n)
&= \mathbb{E}_{\mathbf{Z}|\mathbf{X},\theta^{(t)}} [\log L(\theta;\mathbf{x},\mathbf{Z}) ] \\
&= \mathbb{E}_{\mathbf{Z}|\mathbf{X},\theta^{(t)}} [\log \prod_{i=1}^{n}L(\theta;\mathbf{x}_i,\mathbf{z}_i) ] \\
&= \mathbb{E}_{\mathbf{Z}|\mathbf{X},\theta^{(t)}} [\sum_{i=1}^n \log L(\theta;\mathbf{x}_i,\mathbf{z}_i) ] \\
&= \sum_{i=1}^n\mathbb{E}_{\mathbf{Z}|\mathbf{X};\theta^{(t)}} [\log L(\theta;\mathbf{x}_i,\mathbf{z}_i) ] \\
&= \sum_{i=1}^n \sum_{j\in\{A,B\}} P(Z_i =j | X_i = \mathbf{x}_i; \theta^{(t)}) \log L(\theta_j;\mathbf{x}_i,\mathbf{z}_i) 
\end{align}$$

- *Second*, find the value of $\theta$ that maximizes ${\displaystyle Q(\theta | \theta^{(t)})}$   (**M-step**)

$$\theta^{(t+1)} = \underset{\boldsymbol\theta}{\operatorname{arg\,max}} \ Q(\theta|\theta^{(t)})$$

This yields to a new parameter estimate $\theta^{(t+1)}$ that is a lower bound of the log-likelihood but touches the log-likelihood function at this new $\theta$. And repeat these two steps process until convergence occurs - at this point, the maxima of the lower bound and likelihood functions are the same and we have found the maximum log-likelihood. 

Therefore, in the **E-step**, we identify a function which is a lower bound for the log-likelikelihood, but how do we choose the distribution $Q_i$? We want the $Q$ function to touch the log-likelihood, and know that Jensen’s inequality is an equality only if the function is constant. So $Q_i$ is just the posterior distribution of $z_i$, and this completes the **E-step**.

In the **M-step**, we find the value of $\theta$ that maximizes the $Q$ function, and then we iterate over the $E$ and $M$ steps until convergence.

In [8]:
experiment = np.array([(5,5), (9,1), (8,2), (4,6), (7,3)])
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])

tol = 0.01
max_iter = 100

ll_old = 0
for i in range(max_iter):
    zs_A = []
    zs_B = []

    vs_A = []
    vs_B = []

    ll_new = 0

    # E-step: calculate probability distributions over possible completions
    for x in experiment:

        # multinomial (binomial) log-likelihood
        ll_A = np.sum([x*np.log(thetas[0])])
        ll_B = np.sum([x*np.log(thetas[1])])

        denom = np.exp(ll_A) + np.exp(ll_B)
        z_A = np.exp(ll_A)/denom
        z_B = np.exp(ll_B)/denom

        zs_A.append(z_A)
        zs_B.append(z_B)

        # used for calculating theta
        vs_A.append(np.dot(z_A, x))
        vs_B.append(np.dot(z_B, x))

        # update complete log likelihood
        ll_new += z_A * ll_A + z_B * ll_B

    # M-step: update values for parameters given current distribution
    thetas[0] = np.sum(vs_A, 0)/np.sum(vs_A)
    thetas[1] = np.sum(vs_B, 0)/np.sum(vs_B)
    
    # print distribution of z for each x and current parameter estimate
    print(' ')
    print('Iteration: {}'.format(i+1))
    print("theta_A = {0}, \
          theta_B = {1}, \
          ll = {2}".format(round(thetas[0,0],3),
                           round(thetas[1,0],3),
                           round(ll_new,3)))

    if np.abs(ll_new - ll_old) < tol:
        break
    ll_old = ll_new

 
Iteration: 1
theta_A = 0.713,           theta_B = 0.581,           ll = -32.687
 
Iteration: 2
theta_A = 0.745,           theta_B = 0.569,           ll = -31.259
 
Iteration: 3
theta_A = 0.768,           theta_B = 0.55,           ll = -30.76
 
Iteration: 4
theta_A = 0.783,           theta_B = 0.535,           ll = -30.331
 
Iteration: 5
theta_A = 0.791,           theta_B = 0.526,           ll = -30.071
 
Iteration: 6
theta_A = 0.795,           theta_B = 0.522,           ll = -29.95
 
Iteration: 7
theta_A = 0.796,           theta_B = 0.521,           ll = -29.901
 
Iteration: 8
theta_A = 0.796,           theta_B = 0.52,           ll = -29.881
 
Iteration: 9
theta_A = 0.797,           theta_B = 0.52,           ll = -29.874
